## Start model training:

- Simple classifier that uses cropped images from detectron2
- Even the images that are not cropped 

In [1]:
import torch
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F
import torchvision.transforms as transforms
import torch.optim 
from torchvision import datasets
from torch.autograd import Variable
import numpy as np
import pandas as pd
import cv2
import argparse
import os
from PIL import Image, ImageEnhance, ImageOps
from tqdm import tqdm
import random

In [2]:
load_dir = '../Global_embeddings'
#data = "../cropped_bird_dataset"
batch_size = 32
epochs = 150
lr = 0.01
momentum = 0.9
weight_decay = 3e-3
grad_clip = 5.
seed = 0
use_cuda = False
experiment='../experiment'

#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = "cpu"

torch.manual_seed(seed)

In [3]:
# Features and labels
features_train = torch.load(os.path.join(load_dir, "birds_features_train.pt"), map_location=torch.device(device))
labels_train = torch.load(os.path.join(load_dir, "birds_labels_train.pt"), map_location=torch.device(device))

features_val = torch.load(os.path.join(load_dir, "birds_features_val.pt"), map_location=torch.device(device))
labels_val = torch.load(os.path.join(load_dir, "birds_labels_val.pt"), map_location=torch.device(device))

# Dataloaders
features_tensor = torch.stack([i for i in features_train])
labels_tensor = torch.stack([i for i in labels_train])
train_data = torch.utils.data.TensorDataset(features_tensor, labels_tensor) 

features_tensor = torch.stack([torch.Tensor(i) for i in features_val])
labels_tensor = torch.stack([i for i in labels_val])
val_data = torch.utils.data.TensorDataset(features_tensor,labels_tensor)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=0)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=batch_size, shuffle=False, num_workers=0)

In [4]:
# Model
class Classifier(nn.Module):
    def __init__(self,embedding_dim):
        super(Classifier, self).__init__()
        self.fc1 = nn.Linear(embedding_dim, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 20)

    def forward(self, x):
        x = F.leaky_relu(self.fc1(x))
        x = F.leaky_relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = Classifier(features_train[0].shape[0])

if use_cuda:
    print('Using GPU')
    model.cuda()
else:
    print('Using CPU')

# Optimizer, LR, and criterion
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)

#optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)
criterion = torch.nn.CrossEntropyLoss(reduction="mean")

# Training functions
def train_classifier(model, train_loader, optimizer, lr_scheduler, criterion, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        if use_cuda:
            data, target = Variable(data.cuda()), Variable(target.cuda().long())
                
        else:
              data, target = Variable(data), Variable(target.long())
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
        optimizer.step()
        lr_scheduler.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data.item()))

def validation_classifier(model, criterion, val_loader):
    model.eval()
    validation_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in val_loader:
            if use_cuda:
                data, target = Variable(data.cuda()), Variable(target.cuda().long())
            else:
                data, target = Variable(data), Variable(target.long())
            output = model(data)
                    
            # sum up batch loss
            validation_loss += criterion(output, target).data.item()
            
            # get the index of the max log-probability
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    validation_loss /= len(val_loader.dataset)
    print('\nValidation set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        validation_loss, correct, len(val_loader.dataset),
        100. * correct / len(val_loader.dataset)))
    return(100. * correct / len(val_loader.dataset))

Using CPU


In [5]:
# Training the classifier 
for epoch in range(1, epochs + 1):
    train_classifier(model, train_loader, optimizer, lr_scheduler, criterion, epoch)
    val_acc=validation_classifier(model, criterion, val_loader)
    if val_acc>=93:
      # Save only when it is good enough
        model_file = experiment + '/model_Inceptionv3_' + str(epoch) + '.pth'
        torch.save(model.state_dict(), model_file)
        print('Saved model to ' + model_file + '. You can run `python evaluate.py --model ' + model_file + '` to generate the Kaggle formatted csv file\n')

Train Epoch: 1 [0/1082 (0%)]	Loss: 2.984128
Train Epoch: 1 [320/1082 (29%)]	Loss: 2.931041
Train Epoch: 1 [640/1082 (59%)]	Loss: 2.884696
Train Epoch: 1 [960/1082 (88%)]	Loss: 2.659421

Validation set: Average loss: 0.0985, Accuracy: 62/103 (60%)
Train Epoch: 2 [0/1082 (0%)]	Loss: 2.466790
Train Epoch: 2 [320/1082 (29%)]	Loss: 2.073416
Train Epoch: 2 [640/1082 (59%)]	Loss: 1.830449
Train Epoch: 2 [960/1082 (88%)]	Loss: 1.225646

Validation set: Average loss: 0.0502, Accuracy: 73/103 (71%)
Train Epoch: 3 [0/1082 (0%)]	Loss: 1.461188
Train Epoch: 3 [320/1082 (29%)]	Loss: 1.194447
Train Epoch: 3 [640/1082 (59%)]	Loss: 1.164091
Train Epoch: 3 [960/1082 (88%)]	Loss: 0.828402

Validation set: Average loss: 0.0345, Accuracy: 76/103 (74%)
Train Epoch: 4 [0/1082 (0%)]	Loss: 0.916277
Train Epoch: 4 [320/1082 (29%)]	Loss: 0.814714
Train Epoch: 4 [640/1082 (59%)]	Loss: 0.701625
Train Epoch: 4 [960/1082 (88%)]	Loss: 0.698626

Validation set: Average loss: 0.0296, Accuracy: 85/103 (83%)
Train Epoch:

In [6]:
break

SyntaxError: 'break' outside loop (668683560.py, line 1)

In [ ]:
# Test with Test Time Augmentation

# Test features
features_test = torch.load(os.path.join(load_dir, 'birds_features_test.pt'), map_location=torch.device(device))
features_tensor = torch.stack([i for i in features_test])

test_path_filepath = "../Inceptionv3_embeddings/test_paths.txt"
#test_path_filepath = "../experiment/test_paths.txt"
with open(test_path_filepath, "r") as file:
    test_paths = file.read().split("\n")
    
best_model_path = "../experiment/model_Inceptionv3_107.pth"

# Loading trained model
state_dict = torch.load(best_model_path)
model = Classifier(features_test[0].shape[0])
model.load_state_dict(state_dict)
model.eval()

In [ ]:
if use_cuda:
    print('Using GPU')
    model.cuda()
else:
    print('Using CPU')

output_file = "../experiment/Inceptionv3_kaggle.csv"

with open(output_file, "w") as file:
    file.write("Id,Category\n")
    with torch.no_grad():
        for path, embedding in tqdm(zip(test_paths, features_tensor)):
            if use_cuda:
                embedding = embedding.cuda()
            output = model(embedding)

            output = norm(model(data))
            output_flip = norm(model(data_flip))
            # max TTA
            confidence1, pred1 = torch.max(output.data, 0)
            confidence2, pred2 = torch.max(output_flip.data, 0)
            index = torch.max(torch.stack((confidence1,confidence2)),0)[1].cpu().data.numpy().item(0)
            pred = torch.stack((pred1,pred2))[index]

            pred = output.data.max(0, keepdim=True)[1]
            file.write("%s,%d\n" % (path, pred))
    print(
        "Succesfully wrote "
        + output_file
        + ", you can upload this file to the kaggle competition website"
    )